In this example kernel, we try to demonstrate the LP for the Sudoku game. To study the problem 

$$\min_{X} \|X\|_{L^1} $$
subject to equality constraint $AX = B$.

In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as scs # sparse matrix construction 
import scipy.linalg as scl # linear algebra algorithms
import scipy.optimize as sco # for minimization use
import matplotlib.pylab as plt # for visualization
import time
from collections import defaultdict
import math

print(os.listdir("input"))

['.DS_Store']


In [4]:
small1 = pd.read_csv('input/small1.csv')
small2 = pd.read_csv('input/small2.csv')
large1 = pd.read_csv('input/large1.csv')
large2 = pd.read_csv('input/large2.csv')

In [5]:
def selected_new_quiz(result, clue, head=True):
    new_clue = clue.copy()
    coord = np.nonzero(result != new_clue)
    if head:
        select_clue = result[coord[0][0],coord[1][0]]
        new_clue[coord[0][0],coord[1][0]] = select_clue
    else:
        select_clue = result[coord[0][-1],coord[1][-1]]
        new_clue[coord[0][-1],coord[1][-1]] = select_clue 
    return new_clue

In [6]:
def clue_recovery(result, clue):
    mask = result != clue
    return mask * clue + result

In [7]:
def str2mat(string, N=9):
    N = 9
    mat = np.reshape([int(c) for c in string], (N,N))
    return mat

In [8]:
def mat2str(mat):
    string = ''
    for element in mat.reshape(-1):
        string += str(element)
    return string

In [9]:
def dup_val(arr):
    u, c = np.unique(arr, return_counts=True)
    return u[c>1]

In [10]:
def del_dup(X):
    '''
    set all contracdicted values to 0
    '''
    mat = X.copy()
    location = set()
    M = 3
    for i in range(mat.shape[0]):

        dups = dup_val(mat[i,:])
        if dups.size>0:
            for dup in dups:
                for coord in np.nonzero((mat[i,:]==dup))[0]:
                    location.add((i,coord))

        dups = dup_val(mat[:,i])
        if dups.size>0:
            for dup in dups:
                for coord in np.nonzero((mat[:,i]==dup))[0]:
                    location.add((coord,i))
                    
    for i in range(M):
        for j in range(M):
            dups = dup_val(mat[i*M:i*M+M,j*M:j*M+M])
            if dups.size>0:
                for dup in dups:
                    coord = np.nonzero(mat[i*M:i*M+M,j*M:j*M+M]==dup)
                    for x, y in zip(coord[0],coord[1]):
                        location.add((x+i*M,y+j*M))
    for loc in location:
        mat[loc]=0
    return mat

In [11]:

def fixed_constraints(N=9):
    rowC = np.zeros(N)
    rowC[0] =1
    rowR = np.zeros(N)
    rowR[0] =1
    row = scl.toeplitz(rowC, rowR)
    ROW = np.kron(row, np.kron(np.ones((1,N)), np.eye(N)))
    
    colR = np.kron(np.ones((1,N)), rowC)
    col  = scl.toeplitz(rowC, colR)
    COL  = np.kron(col, np.eye(N))
    
    M = int(np.sqrt(N))
    boxC = np.zeros(M)
    boxC[0]=1
    boxR = np.kron(np.ones((1, M)), boxC) 
    box = scl.toeplitz(boxC, boxR)
    box = np.kron(np.eye(M), box)
    BOX = np.kron(box, np.block([np.eye(N), np.eye(N) ,np.eye(N)]))
    
    cell = np.eye(N**2)
    CELL = np.kron(cell, np.ones((1,N)))
    
    return scs.csr_matrix(np.block([[ROW],[COL],[BOX],[CELL]]))




# For the constraint from clues, we extract the nonzeros from the quiz string.
def clue_constraint(m, N=9):
    r, c = np.where(m.T)
    v = np.array([m[c[d],r[d]] for d in range(len(r))])
    
    table = N * c + r
    table = np.block([[table],[v-1]])
    
    # it is faster to use lil_matrix when changing the sparse structure.
    CLUE = scs.lil_matrix((len(table.T), N**3))
    for i in range(len(table.T)):
        CLUE[i,table[0,i]*N + table[1,i]] = 1
    # change back to csr_matrix.
    CLUE = CLUE.tocsr() 
    
    return CLUE

In [12]:
def solve(quiz,string=False):
    
    A0 = fixed_constraints()
    if string:
        A1 = clue_constraint(str2mat(quiz))
    else :
        A1 = clue_constraint(quiz)
    # Formulate the matrix A and vector B (B is all ones).
    A = scs.vstack((A0,A1))
    A = A.toarray()
    B = np.ones(A.shape[0])


    # Because rank defficiency. We need to extract effective rank.
    u, s, vh = np.linalg.svd(A, full_matrices=False)
    K = np.sum(s > 1e-12)
    S = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
    A = S@vh
    B = u.T@B
    B = B[:K]

    c = np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ])
    G = np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                         [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]])
    h = np.zeros(A.shape[1]*2)
    H = np.block([A, -A])
    b = B

    ret = sco.linprog(c, G, h, H, b, method='interior-point', options={'tol':1e-6})
    x = ret.x[:A.shape[1]] - ret.x[A.shape[1]:]

    
    z = np.reshape(x, (81, 9))
    result = np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) )
    return result

In [20]:
def weighted_solve(quiz, eps=10, L=10, string=False):
    tol = 1e-10
    
    A0 = fixed_constraints()
    if string:
        A1 = clue_constraint(str2mat(quiz))
    else :
        A1 = clue_constraint(quiz)

    # Formulate the matrix A and vector B (B is all ones).
    A = scs.vstack((A0,A1))
    A = A.toarray()
    B = np.ones(A.shape[0])

    # Because rank defficiency. We need to extract effective rank.
    u, s, vh = np.linalg.svd(A, full_matrices=False)
    K = np.sum(s > 1e-12)
    S = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
    A = S@vh
    B = u.T@B
    B = B[:K]

    c = np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ])
    G = np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                         [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]])
    h = np.zeros(A.shape[1]*2)
    H = np.block([A, -A])
    b = B
    ret = sco.linprog(c, G, h, H, b, method='interior-point', options={'tol':1e-6})
    x0 = ret.x[:A.shape[1]] - ret.x[A.shape[1]:]
    for i in range(L):
        W = (1/(np.abs(x0)+eps))
        c = np.concatenate((W,W),axis=0)
        ret = sco.linprog(c, G, h, H, b, method='interior-point', options={'tol':1e-6})
        x = ret.x[:A.shape[1]] - ret.x[A.shape[1]:]
        if np.linalg.norm(x0-x) < tol:
            break
        x0 = x
    z = np.reshape(x, (81, 9))
    result = np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) )
    return result

In [14]:
def forward_checking(m , N=9):
    for i in range(N):
        for j in range(N):
            if m[i][j] == 0:
                m[i][j] = 123456789
                for k in m[i]:
                    if k < 10:
                        s = str(m[i][j]).replace(str(k), '0')
                        m[i][j] = int(s)
                for k in m[:,j]:
                    if k < 10:
                        s = str(m[i][j]).replace(str(k), '0')
                        m[i][j] = int(s)
                dd = my_block(m) #block dict
                ll = dd[(math.ceil(i/N), math.ceil(j/N))]
                #for k in ll:         #eliminate block
                    #if k < 10:
                        #s = str(m[i][j]).replace(str(k), '0')
                        #m[i][j] = int(s)
                
    return m

def my_block(mygb, N=9):
    dd = defaultdict(list)
    for i in range(len(mygb)):
        for j in range(len(mygb[i])):
            key = (math.ceil(i/N), math.ceil(j/N))
            dd[key].append(mygb[i][j])
    return dd

In [15]:
def clean(x):
    if x > 9:
        x = 0
    return x

In [24]:
def sudoku_solver(quiz, eps=10,L=10):
    clue = str2mat(quiz)
    quiz = forward_checking(str2mat(quiz))
    quiz = np.vectorize(clean)(quiz)
    result = weighted_solve(quiz,eps,L)
    
    
    result_del = del_dup(result)
    if np.any(result_del==0):
#         print('sample:',i,', try step 1')
        quiz_2 = clue_recovery(result_del,clue)
        result = weighted_solve(quiz_2,eps,L)
        result_del = del_dup(result)
        if np.any(result_del==0):
#             print('sample:',i,', try step 2')
            quiz_2 = clue_recovery(result_del,clue)
            result = weighted_solve(quiz_2,eps,L)
            result_del = del_dup(result)
            if np.any(result_del==0):
#                 print('sample:',i,', try step 3 head')
                quiz_3 = selected_new_quiz(result_del, clue)
                result = weighted_solve(quiz_2,eps,L)
                result_del = del_dup(result)
                if np.any(result_del==0):
#                     print('sample:',i,', try step 3 tail')
                    quiz_3 = selected_new_quiz(result_del, clue, head=False)
                    result = weighted_solve(quiz_2,eps,L)
    return result

### small1

In [17]:
corr_cnt = 0
start = time.time()
data = small1


for i in range(data.shape[0]):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    
    
    result = sudoku_solver(quiz)

    if np.linalg.norm( result - str2mat(solu), np.inf) >0:
#         print('sample:',i,' all steps failed\n')

        pass
    else:
#         print('sample:', i,'succeed\n')
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   0.37 secs. Success rate: 20 / 20 
Aver Time:   0.37 secs. Success rate: 24 / 24 


### small2

In [18]:
corr_cnt = 0
start = time.time()
data = small2

for i in range(data.shape[0]):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    
    
    result = sudoku_solver(quiz)

    if np.linalg.norm( result - str2mat(solu), np.inf) >0:
#         print('sample:',i,' all steps failed\n')

        pass
    else:
#         print('sample:', i,'succeed\n')
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   0.53 secs. Success rate: 19 / 20 
Aver Time:   0.73 secs. Success rate: 36 / 40 
Aver Time:   0.85 secs. Success rate: 52 / 60 
Aver Time:   0.86 secs. Success rate: 70 / 80 
Aver Time:   0.91 secs. Success rate: 87 / 100 
Aver Time:   0.91 secs. Success rate: 105 / 120 
Aver Time:   1.07 secs. Success rate: 116 / 140 
Aver Time:   1.17 secs. Success rate: 129 / 160 
Aver Time:   1.20 secs. Success rate: 144 / 180 
Aver Time:   1.25 secs. Success rate: 157 / 200 
Aver Time:   1.27 secs. Success rate: 171 / 220 
Aver Time:   1.32 secs. Success rate: 183 / 240 
Aver Time:   1.32 secs. Success rate: 199 / 260 
Aver Time:   1.37 secs. Success rate: 213 / 280 
Aver Time:   1.40 secs. Success rate: 226 / 300 
Aver Time:   1.46 secs. Success rate: 236 / 320 
Aver Time:   1.50 secs. Success rate: 246 / 340 
Aver Time:   1.46 secs. Success rate: 265 / 360 
Aver Time:   1.48 secs. Success rate: 278 / 380 
Aver Time:   1.52 secs. Success rate: 287 / 400 
Aver Time:   1.55 secs. Succe

In [26]:
corr_cnt = 0
start = time.time()
data = small2

for i in range(data.shape[0]):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    
    
    result = sudoku_solver(quiz,L=30)

    if np.linalg.norm( result - str2mat(solu), np.inf) >0:
#         print('sample:',i,' all steps failed\n')

        pass
    else:
#         print('sample:', i,'succeed\n')
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   0.75 secs. Success rate: 19 / 20 
Aver Time:   0.92 secs. Success rate: 36 / 40 
Aver Time:   1.00 secs. Success rate: 52 / 60 
Aver Time:   0.97 secs. Success rate: 70 / 80 
Aver Time:   0.99 secs. Success rate: 87 / 100 
Aver Time:   0.98 secs. Success rate: 105 / 120 
Aver Time:   1.13 secs. Success rate: 116 / 140 
Aver Time:   1.21 secs. Success rate: 129 / 160 
Aver Time:   1.23 secs. Success rate: 144 / 180 
Aver Time:   1.29 secs. Success rate: 157 / 200 
Aver Time:   1.31 secs. Success rate: 171 / 220 
Aver Time:   1.36 secs. Success rate: 183 / 240 
Aver Time:   1.35 secs. Success rate: 199 / 260 
Aver Time:   1.37 secs. Success rate: 213 / 280 
Aver Time:   1.39 secs. Success rate: 226 / 300 
Aver Time:   1.44 secs. Success rate: 236 / 320 
Aver Time:   1.49 secs. Success rate: 246 / 340 
Aver Time:   1.45 secs. Success rate: 265 / 360 
Aver Time:   1.46 secs. Success rate: 278 / 380 
Aver Time:   1.51 secs. Success rate: 287 / 400 
Aver Time:   1.54 secs. Succe

### other $\epsilon$ test, defalt $=10$

In [25]:
corr_cnt = 0
start = time.time()
data = small2

for i in range(data.shape[0]):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    
    
    result = sudoku_solver(quiz,eps=20)

    if np.linalg.norm( result - str2mat(solu), np.inf) >0:
#         print('sample:',i,' all steps failed\n')

        pass
    else:
#         print('sample:', i,'succeed\n')
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   0.53 secs. Success rate: 19 / 20 
Aver Time:   0.61 secs. Success rate: 37 / 40 
Aver Time:   0.71 secs. Success rate: 53 / 60 
Aver Time:   0.71 secs. Success rate: 71 / 80 
Aver Time:   0.71 secs. Success rate: 89 / 100 
Aver Time:   0.70 secs. Success rate: 107 / 120 
Aver Time:   0.81 secs. Success rate: 119 / 140 
Aver Time:   0.86 secs. Success rate: 132 / 160 
Aver Time:   0.88 secs. Success rate: 147 / 180 
Aver Time:   0.92 secs. Success rate: 160 / 200 
Aver Time:   0.93 secs. Success rate: 175 / 220 
Aver Time:   0.98 secs. Success rate: 186 / 240 
Aver Time:   0.97 secs. Success rate: 202 / 260 
Aver Time:   0.98 secs. Success rate: 216 / 280 
Aver Time:   1.00 secs. Success rate: 230 / 300 
Aver Time:   1.04 secs. Success rate: 240 / 320 
Aver Time:   1.07 secs. Success rate: 249 / 340 
Aver Time:   1.05 secs. Success rate: 268 / 360 
Aver Time:   1.05 secs. Success rate: 282 / 380 
Aver Time:   1.09 secs. Success rate: 291 / 400 
Aver Time:   1.10 secs. Succe

### $\epsilon=30,40$ SVD did not converge 

In [26]:
corr_cnt = 0
start = time.time()
data = small2

for i in range(data.shape[0]):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    
    
    result = sudoku_solver(quiz,eps=30)

    if np.linalg.norm( result - str2mat(solu), np.inf) >0:
#         print('sample:',i,' all steps failed\n')

        pass
    else:
#         print('sample:', i,'succeed\n')
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   0.55 secs. Success rate: 19 / 20 
Aver Time:   0.67 secs. Success rate: 36 / 40 
Aver Time:   0.75 secs. Success rate: 52 / 60 
Aver Time:   0.74 secs. Success rate: 70 / 80 
Aver Time:   0.77 secs. Success rate: 87 / 100 
Aver Time:   0.75 secs. Success rate: 105 / 120 
Aver Time:   0.88 secs. Success rate: 116 / 140 
Aver Time:   0.93 secs. Success rate: 129 / 160 
Aver Time:   0.94 secs. Success rate: 145 / 180 
Aver Time:   0.97 secs. Success rate: 159 / 200 
Aver Time:   0.99 secs. Success rate: 173 / 220 
Aver Time:   1.03 secs. Success rate: 185 / 240 
Aver Time:   1.02 secs. Success rate: 201 / 260 
Aver Time:   1.04 secs. Success rate: 215 / 280 
Aver Time:   1.06 secs. Success rate: 228 / 300 
Aver Time:   1.10 secs. Success rate: 238 / 320 
Aver Time:   1.13 secs. Success rate: 248 / 340 
Aver Time:   1.11 secs. Success rate: 266 / 360 
Aver Time:   1.12 secs. Success rate: 279 / 380 
Aver Time:   1.15 secs. Success rate: 288 / 400 
Aver Time:   1.18 secs. Succe

LinAlgError: SVD did not converge

### large1

In [19]:
corr_cnt = 0
start = time.time()
data = large1
random_seed = 42
np.random.seed(random_seed)
samples = np.random.choice(len(data), 1000)
for i in range(len(samples)):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    
    
    result = sudoku_solver(quiz)

    if np.linalg.norm( result - str2mat(solu), np.inf) >0:
#         print('sample:',i,' all steps failed\n')

        pass
    else:
#         print('sample:', i,'succeed\n')
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   0.50 secs. Success rate: 19 / 20 
Aver Time:   0.44 secs. Success rate: 39 / 40 
Aver Time:   0.43 secs. Success rate: 59 / 60 
Aver Time:   0.45 secs. Success rate: 78 / 80 
Aver Time:   0.43 secs. Success rate: 98 / 100 
Aver Time:   0.43 secs. Success rate: 118 / 120 
Aver Time:   0.42 secs. Success rate: 138 / 140 
Aver Time:   0.43 secs. Success rate: 157 / 160 
Aver Time:   0.44 secs. Success rate: 176 / 180 
Aver Time:   0.44 secs. Success rate: 196 / 200 
Aver Time:   0.46 secs. Success rate: 214 / 220 
Aver Time:   0.45 secs. Success rate: 234 / 240 
Aver Time:   0.46 secs. Success rate: 253 / 260 
Aver Time:   0.46 secs. Success rate: 272 / 280 
Aver Time:   0.47 secs. Success rate: 291 / 300 
Aver Time:   0.46 secs. Success rate: 311 / 320 
Aver Time:   0.48 secs. Success rate: 328 / 340 
Aver Time:   0.48 secs. Success rate: 348 / 360 
Aver Time:   0.49 secs. Success rate: 366 / 380 
Aver Time:   0.48 secs. Success rate: 386 / 400 
Aver Time:   0.48 secs. Succe

### large2

In [20]:
corr_cnt = 0
start = time.time()
data = large2
random_seed = 42
np.random.seed(random_seed)
samples = np.random.choice(len(data), 1000)
for i in range(len(samples)):
    quiz = data["quizzes"][i]
    solu = data["solutions"][i]
    
    
    result = sudoku_solver(quiz)

    if np.linalg.norm( result - str2mat(solu), np.inf) >0:
#         print('sample:',i,' all steps failed\n')

        pass
    else:
#         print('sample:', i,'succeed\n')
        corr_cnt += 1
    
    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

Aver Time:   0.38 secs. Success rate: 20 / 20 
Aver Time:   0.38 secs. Success rate: 40 / 40 
Aver Time:   0.38 secs. Success rate: 60 / 60 
Aver Time:   0.38 secs. Success rate: 80 / 80 
Aver Time:   0.39 secs. Success rate: 100 / 100 
Aver Time:   0.39 secs. Success rate: 120 / 120 
Aver Time:   0.39 secs. Success rate: 140 / 140 
Aver Time:   0.39 secs. Success rate: 160 / 160 
Aver Time:   0.39 secs. Success rate: 180 / 180 
Aver Time:   0.39 secs. Success rate: 200 / 200 
Aver Time:   0.39 secs. Success rate: 220 / 220 
Aver Time:   0.39 secs. Success rate: 240 / 240 
Aver Time:   0.39 secs. Success rate: 260 / 260 
Aver Time:   0.39 secs. Success rate: 280 / 280 
Aver Time:   0.39 secs. Success rate: 300 / 300 
Aver Time:   0.39 secs. Success rate: 320 / 320 
Aver Time:   0.39 secs. Success rate: 340 / 340 
Aver Time:   0.39 secs. Success rate: 360 / 360 
Aver Time:   0.40 secs. Success rate: 380 / 380 
Aver Time:   0.40 secs. Success rate: 400 / 400 
Aver Time:   0.40 secs. Succ